In [81]:
!pip install vaderSentiment

In [82]:
!pip install ktrain

In [4]:
!pip install langid

In [5]:
!pip install vaderSentiment

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import string
import re
import langid
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import ktrain
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [8]:
df_dota = pd.read_csv("/kaggle/input/gosuai-dota-2-game-chats/dota2_chat_messages.csv")

In [9]:
df_league_of_legends = pd.read_csv("/kaggle/input/league-of-legends-tribunal-chatlogs/chatlogs.csv")

Getting column number of column containing chat text in both dataset

In [10]:
df_dota.columns.get_loc('text')

3

In [11]:
df_league_of_legends.columns.get_loc('message')

1

In [13]:
df_train = pd.DataFrame(columns = ["chat", "sentiment"])

In [14]:
df_train["chat"] = pd.concat([df_dota[df_dota.columns[3]], df_league_of_legends[df_league_of_legends.columns[1]]], ignore_index=True)

In [15]:
df_train = df_train.sample(n=150000)

In [16]:
df_train.describe()

,chat,sentiment
count,150000,0
unique,93452,0
top,gg,NaN
freq,4230,NaN


In [17]:
len(df_train)

150000

Removing digits, punctuation, URL and email.

In [19]:
def remove_digits(text):
    return ''.join([char for char in str(text) if not char.isdigit()])

def remove_punctuation(text):
    return ''.join([char for char in str(text) if char not in string.punctuation])

def remove_email_address(text):
    return re.sub("\S*@\S*\s?", "", str(text))

def remove_url(text):
    return re.sub("http\S+", "", str(text))

def preprocess_text(dataframe_column):
    dataframe_column = dataframe_column.map(lambda text : remove_digits(text))
    dataframe_column = dataframe_column.map(lambda text : remove_punctuation(text))
    dataframe_column = dataframe_column.map(lambda text : remove_email_address(text))
    dataframe_column = dataframe_column.map(lambda text : remove_url(text))
    return dataframe_column

In [20]:
df_train['chat'] = preprocess_text(df_train['chat'])

In [21]:
df_train_eng = pd.DataFrame(columns = ["chat", "sentiment"])

In [22]:
eng_chat = []

In [23]:
for index, row in df_train.iterrows():
    if langid.classify(row["chat"])[0] == 'en':
        eng_chat.append(row["chat"])

In [24]:
len(eng_chat)

81335

In [25]:
df_train_eng['chat'] = eng_chat

In [26]:
df_train_eng.describe()

,chat,sentiment
count,81335,0
unique,36659,0
top,,NaN
freq,7965,NaN


In [27]:
df_train_eng = df_train_eng.drop_duplicates(subset=['chat'], keep='first')

In [28]:
df_train_eng.describe()

,chat,sentiment
count,36659,0
unique,36659,0
top,USELESS ALCHE I EVER SAW,NaN
freq,1,NaN


In [29]:
df_train_eng = df_train_eng[df_train_eng['chat'].notna()]

In [30]:
df_train_eng['chat'].replace('', np.nan, inplace=True)

No null row now

In [31]:
df_train_eng.describe()

,chat,sentiment
count,36658,0
unique,36658,0
top,USELESS ALCHE I EVER SAW,NaN
freq,1,NaN


In [32]:
df_train_eng.reset_index(drop=True, inplace = True)

In [33]:
sentiment_analyzer = SentimentIntensityAnalyzer()

In [34]:
def build_row(row):
    compound_score = sentiment_analyzer.polarity_scores(str(row['chat']))['compound']
    if compound_score <= -0.05:
        senti_score = 0
    elif compound_score > -0.05 and compound_score < 0.05:
        senti_score = 1
    elif compound_score >= 0.05:
        senti_score = 2
    return senti_score

In [35]:
df_train_eng['sentiment'] = df_train_eng.apply(lambda row: build_row(row), axis = 1)

In [38]:
len(df_train_eng)

36659

In [39]:
x_train = df_train_eng['chat'][0:24439].values.tolist()
y_train = df_train_eng['sentiment'][0:24439].values.tolist()
x_test = df_train_eng['chat'][24439:].values.tolist()
y_test = df_train_eng['sentiment'][24439:].values.tolist()

In [41]:
print(len(x_train), len(x_train), len(x_test), len(y_test))

24439 24439 12220 12220


In [42]:
(x_train,  y_train), (x_test, y_test), preproc = ktrain.text.texts_from_array(x_train=x_train, y_train=y_train,                                                                    
                                                                              x_test=x_test, y_test=y_test,                                                                              
                                                                              class_names=['0','1', '2'],
                                                                              maxlen=128,
                                                                              preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [43]:
model = ktrain.text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=32)

Is Multi-Label? False
maxlen is 128


/opt/conda/lib/python3.10/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [44]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input-Token (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 Input-Segment (InputLayer)     [(None, 128)]        0           []                               
                                                                                                  
 Embedding-Token (TokenEmbeddin  [(None, 128, 768),  23440896    ['Input-Token[0][0]']            
 g)                              (30522, 768)]                                                    
                                                                                                  
 Embedding-Segment (Embedding)  (None, 128, 768)     1536        ['Input-Segment[0][0]']    

In [45]:
result = learner.fit_onecycle(3e-5, 4)



begin training using onecycle policy with max lr of 3e-05...
Epoch 1/4
764/764 [==============================] - 639s 802ms/step - loss: 0.4463 - accuracy: 0.8209
Epoch 2/4
764/764 [==============================] - 616s 806ms/step - loss: 0.1267 - accuracy: 0.9608
Epoch 3/4
764/764 [==============================] - 616s 806ms/step - loss: 0.0546 - accuracy: 0.9833
Epoch 4/4
764/764 [==============================] - 615s 806ms/step - loss: 0.0213 - accuracy: 0.9934


In [46]:
validate = learner.validate(val_data=(x_test, y_test), class_names=['0', '1', '2'])

382/382 [==============================] - 109s 275ms/step
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      2238
           1       0.99      0.99      0.99      7390
           2       0.97      0.96      0.96      2592

    accuracy                           0.98     12220
   macro avg       0.97      0.97      0.97     12220
weighted avg       0.98      0.98      0.98     12220



In [47]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [48]:
predictor.get_classes()

['0', '1', '2']

In [50]:
prediction = predictor.predict(df_train_eng['chat'][24439:].tolist())

In [61]:
df_train_eng['sentiment'][24439:].describe()

count    12220.000000
mean         1.028969
std          0.628050
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
Name: sentiment, dtype: float64

In [75]:
print(type(prediction[0]))

<class 'str'>


In [77]:
prediction = [int(i) for i in prediction]

In [79]:
print(mean_squared_error(df_train_eng['sentiment'][24439:], prediction))

0.044271685761047466


In [80]:
print(mean_absolute_error(df_train_eng['sentiment'][24439:], prediction))

0.029869067103109655
